# Some Installs

In [1]:
!pip install  -q -U langgraph "langchain[openai]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.7/216.7 kB 17.3 MB/s eta 0:00:00


In [2]:
! pip install -q gradio

In [3]:
! pip install -q langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


# Imports

In [4]:

from typing import Annotated, TypedDict, List, Dict, Any, Optional
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_community.agent_toolkits import PlayWrightBrowserToolkit
from langchain_community.tools.playwright.utils import create_async_playwright_browser
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import ToolNode,tools_condition
from langgraph.graph.message import add_messages
from pydantic import BaseModel, Field
from IPython.display import Image, display
import gradio as gr
import uuid
from dotenv import load_dotenv
import os
from langchain.agents import tool,Tool

# Let's Start

In [5]:
# LangSmith
f=open("/content/LangSmith API.txt","r")
smith_api_key=f.read()
os.environ["LANGCHAIN_API_KEY"] = smith_api_key
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "My-Research-Assistant"

# OpenAI Key
f=open("/content/OpenAIKey.txt","r")
api_key=f.read()



* The LLM I Will Use

In [6]:
llm = ChatOpenAI(
    model_name="x-ai/grok-4-fast:free",
    openai_api_key=api_key,
    openai_api_base="https://openrouter.ai/api/v1",
    temperature=0.0)

* I will use one node in order to create the Researchers

In [7]:
class Researchers(BaseModel):
  name: str= Field(description='The name of the researcher')
  role: str= Field(description='The role of the researcher in the context of the topic')
  research_interests: str= Field(description='The research interests of the researcher')
  research_field: str= Field(description='The research field of the researcher')
  @property
  def description(self)->str:
    return f"Name: {self.name}\nRole: {self.role}\nResearch Interests: {self.research_interests}\nResearch Field: {self.research_field}"

In [8]:
class Res_List(BaseModel):
  re_list: List[Researchers]= Field(description='The list of researchers with role, name, research interests and research field')

In [9]:
class Re_State(TypedDict):
  re_list: List[Researchers]
  topic: str
  max_researchers:int



In [ ]:
def create_researchers(state:Re_State)->Res_List:
  llm_with_structure=llm.with_structured_output(Res_List)

  prompt=ChatPromptTemplate.from_messages([
      ('system',"""You are tasked with creating a set of AI researchers.
      1. First read carefully the the topic:\n {topic}
      2. Determine the most interesting themes based upon documents or feedback above.
      3. Pick the top {max_researchers} themes."""),
      ('user'," Generate the set of analyst")])